In [42]:
import pandas as pd 
from dotenv import load_dotenv
import os

df = pd.read_csv("eventos_2025.tsv", sep="\t", skiprows=1)

# Display more information about the dataframe
print("DataFrame shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nSample data:")
print(df.head())

# Check if there's an address column
address_columns = [col for col in df.columns if "ENDEREÇO" in col or "ENDERECO" in col or "LOCAL" in col]
if address_columns:
    print("\nAddress column found:", address_columns)
    print("\nSample addresses:")
    print(df[address_columns].head())
else:
    print("\nNo obvious address column found. Here are all columns with their first values:")
    for col in df.columns:
        print(f"{col}: {df[col].iloc[0]}")

DataFrame shape: (22, 10)

Columns: ['DATA', 'INICIO', 'TERMINO', 'EVENTO', 'ATIVIDADE', 'ACESSO', 'LOCAL', 'ENDEREÇO', 'ARTICULAÇÃO', 'CONTATO']

Sample data:
         DATA INICIO TERMINO                      EVENTO  \
0  25/04/2025     9h     16h  Desafio da Natureza Urbana   
1  25/04/2025     9h     12h  Desafio da Natureza Urbana   
2  25/04/2025     9h     11h  Desafio da Natureza Urbana   
3  25/04/2025    10h   11h30  Desafio da Natureza Urbana   
4  25/04/2025    14h   15h30  Desafio da Natureza Urbana   

                                           ATIVIDADE   ACESSO  \
0                                Saída de observação  PÚBLICO   
1                                Saída de observação  PÚBLICO   
2            Observação e sensibilização de natureza  PÚBLICO   
3  Observação e sensibilização de natureza - Alun...  PÚBLICO   
4  Observação e sensibilização de natureza - Alun...  PÚBLICO   

                                        LOCAL  \
0  Parque Estadual Cantareira - Núcleo 

In [43]:
# # IMPORTANT: Please run the installation cell above manually
# # By default, I don't run cells that install packages for safety reasons
# # This default can be changed in settings if needed

# # Let's proceed with the geocoding process once packages are installed
# # This cell will process all addresses and add latitude/longitude columns

# # First, let's define a function to process all addresses
# def process_addresses(dataframe):
#     import pandas as pd
#     try:
#         # Try importing the required packages
#         from geopy.geocoders import Nominatim
#         from geopy.extra.rate_limiter import RateLimiter
#     except ImportError:
#         print("ERROR: Required packages not installed. Please run the installation cell above.")
#         return dataframe
    
#     print("Starting geocoding process. This may take some time...")
    
#     from geopy.geocoders import GoogleV3

#     # Load environment variables from .env file
#     load_dotenv()

#     # Get the API key from the environment variable
#     api_key = os.getenv("GOOGLE_API_KEY")
#     if not api_key:
#         raise ValueError("Google API key not found. Please set it in the .env file.")

#     # Initialize the geolocator with the API key
#     geolocator = GoogleV3(api_key=api_key)

#     # Use rate limiting to avoid being blocked
#     geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0.1)
    
#     # Function to extract coordinates from an address
#     def get_coordinates(address):
#         try:
#             location = geocode(address + ", São Paulo, Brazil")  # Adding São Paulo, Brazil to improve geocoding
#             if location:
#                 return (location.latitude, location.longitude)
#             return (None, None)
#         except Exception as e:
#             print(f"Error geocoding address: {address}. Error: {e}")
#             return (None, None)
    
#     # Create a copy of the dataframe to avoid modifying the original
#     df_with_coords = dataframe.copy()
    
#     # Create new columns for latitude and longitude
#     df_with_coords[['latitude', 'longitude']] = df_with_coords['ENDEREÇO'].apply(lambda x: pd.Series(get_coordinates(x)))
    
#     # Count how many addresses were successfully geocoded
#     success_count = df_with_coords['latitude'].notna().sum()
#     total_count = len(df_with_coords)
    
#     print(f"Geocoding completed: {success_count}/{total_count} addresses successfully geocoded.")
    
#     return df_with_coords

# # Run the function when packages are installed
# try:
#     df_with_coords = process_addresses(df)
#     # Save the dataframe with coordinates to a new TSV file
#     df_with_coords.to_csv("eventos_2025_with_coords.tsv", sep="\t", index=False)
#     print("Data with coordinates saved to 'eventos_2025_with_coords.tsv'")
    
#     # Display the first few rows with coordinates
#     print("\nSample data with coordinates:")
#     print(df_with_coords[['EVENTO', 'DATA', 'ENDEREÇO', 'latitude', 'longitude']].head())
# except NameError as e:
#     print(f"Error: {e}. Please make sure to run the installation cell first.")

In [44]:
# Create an interactive map with the geocoded locations
# This will also generate an HTML file that can be used for GitHub Pages

import folium
import pandas as pd
from datetime import datetime
import os
from icalendar import Calendar, Event
import pytz
import re

# Load the data with coordinates
df_with_coords = pd.read_csv("eventos_2025_with_coords.tsv", sep="\t")


# Filter out rows with missing coordinates
df_mapped = df_with_coords.dropna(subset=['latitude', 'longitude'])
print(f"Creating map with {len(df_mapped)} locations (out of {len(df_with_coords)} total events)")

# Parse dates for filtering
def parse_date(date_str):
    try:
        # Convert Brazilian date format (DD/MM/YYYY) to datetime
        return datetime.strptime(date_str, '%d/%m/%Y')
    except:
        return None

# Add parsed date column
df_mapped['parsed_date'] = df_mapped['DATA'].apply(parse_date)

# Get unique dates for filtering
unique_dates = sorted(df_mapped['parsed_date'].dropna().unique())
date_options = [d.strftime('%d/%m/%Y') for d in unique_dates]
print(f"Available dates: {date_options}")

# Create a base map centered on São Paulo
map_center = [-23.55, -46.63]  # São Paulo coordinates
m = folium.Map(location=map_center, zoom_start=10, tiles="OpenStreetMap")

# Add markers for each location
for idx, row in df_mapped.iterrows():
    # Create popup content with event details
    popup_content = f"""
    <div style="width: 250px">
        <h4>{row['EVENTO']}</h4>
        <p><b>Data:</b> {row['DATA']}</p>
        <p><b>Horário:</b> {row['INICIO']} - {row['TERMINO']}</p>
        <p><b>Atividade:</b> {row['ATIVIDADE']}</p>
        <p><b>Local:</b> {row['LOCAL']}</p>
        <p><b>Endereço:</b> {row['ENDEREÇO']}</p>
        <p><b>Acesso:</b> {row['ACESSO']}</p>
        <p><b>Articulação</b> {row['ARTICULAÇÃO']}</p>
    </div>
    """
    
    # Add marker with popup
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_content, max_width=300),
        tooltip=f"{row['EVENTO']} - {row['DATA']}",
        icon=folium.Icon(color='green', icon='leaf', prefix='fa')
    ).add_to(m)

# Save the map to an HTML file
map_file = "map.html"
m.save(map_file)
print(f"Interactive map saved to {map_file}")

# Now create the main index.html file with date filtering functionality
html_content = """
<!DOCTYPE html>
<html lang="pt-BR">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>City Nature Challenge 2025: São Paulo e Região Metropolitana</title>
    <link rel="stylesheet" href="https://unpkg.com/leaflet@1.7.1/dist/leaflet.css" />
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            line-height: 1.6;
        }
        .container {
            max-width: 1200px;
            margin: 0 auto;
            padding: 20px;
        }
        header {
            background-color: #4CAF50;
            color: white;
            text-align: center;
            padding: 1em;
        }
        #map {
            height: 600px;
            width: 100%;
            margin-bottom: 20px;
        }
        .controls {
            margin-bottom: 20px;
            padding: 10px;
            background-color: #f5f5f5;
            border-radius: 5px;
        }
        .btn {
            background-color: #4CAF50;
            color: white;
            border: none;
            padding: 8px 16px;
            text-align: center;
            text-decoration: none;
            display: inline-block;
            font-size: 14px;
            margin: 4px 2px;
            cursor: pointer;
            border-radius: 4px;
        }
        select {
            padding: 8px;
            border-radius: 4px;
            border: 1px solid #ddd;
        }
        footer {
            background-color: #f5f5f5;
            text-align: center;
            padding: 1em;
            margin-top: 20px;
        }
    </style>
</head>
<body>
    <header>
        <h1>City Nature Challenge 2025: São Paulo e Região Metropolitana</h1>
    </header>
    
    <div class="container">
        <div class="description">
            <p>Bem vindos ao City Nature Challenge 2025, também conhecido como Desafio Natureza nas Cidades! Por mais um ano estaremos competindo com cidades do mundo todo com o intuito de mostrar quão grande é a nossa biodiversidade! Seja nos parques, em trilhas, praças, ruas ou quintais, todos os registros são importantes e bem vindos! Então preparem seus celulares, suas câmeras e seus olhos, pois o desafio vai começar!</p>
        </div>
        
        <div class="controls">
            <label for="date-filter">Filtrar por data: </label>
            <select id="date-filter">
                <option value="all">Todas as datas</option>
                <!-- Date options will be added here by JavaScript -->
            </select>
            <button class="btn" id="reset-filter">Mostrar Todos</button>
            <a href="city_nature_challenge_2025.ics" download class="btn">Baixar Calendário (.ics)</a>
        </div>
        
        <div id="map"></div>
    </div>
    
    <footer>
        <p>&copy; 2025 City Nature Challenge - São Paulo e Região Metropolitana</p>
    </footer>

    <script src="https://unpkg.com/leaflet@1.7.1/dist/leaflet.js"></script>
    <script>
        // Initialize the map
        const map = L.map('map').setView([-23.55, -46.63], 10);
        
        // Add OpenStreetMap tiles
        L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
            attribution: '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'
        }).addTo(map);
        
        // Define date options
        const dateOptions = ["25/04/2025", "26/04/2025"];
        
        // Populate date filter dropdown
        const dateFilter = document.getElementById('date-filter');
        dateOptions.forEach(date => {
            const option = document.createElement('option');
            option.value = date;
            option.textContent = date;
            dateFilter.appendChild(option);
        });
        
        // Define event data
        const events = [
            // Event data will be added here manually to avoid JSON serialization issues
        ];
        
        // Function to safely escape HTML content
        function escapeHtml(unsafe) {
            return unsafe
                .replace(/&/g, "&amp;")
                .replace(/</g, "&lt;")
                .replace(/>/g, "&gt;")
                .replace(/"/g, "&quot;")
                .replace(/'/g, "&#039;");
        }
        
        // Add markers for each event
        const markers = [];
        
        // Add event data and markers
        // This will be populated by Python with actual event data
        
        // Filter events by date
        dateFilter.addEventListener('change', function() {
            const selectedDate = this.value;
            
            markers.forEach(item => {
                if (selectedDate === 'all' || item.date === selectedDate) {
                    map.addLayer(item.marker);
                } else {
                    map.removeLayer(item.marker);
                }
            });
        });
        
        // Reset filter button
        document.getElementById('reset-filter').addEventListener('click', function() {
            dateFilter.value = 'all';
            markers.forEach(item => {
                map.addLayer(item.marker);
            });
        });
    </script>
</body>
</html>
"""

# Prepare event data for JavaScript
event_markers_js = ""
for idx, row in df_mapped.iterrows():
    # Safely handle string values to avoid issues with quotes and special characters
    evento = str(row['EVENTO']).replace('"', '\\"').replace("'", "\\'")
    data = str(row['DATA'])
    inicio = str(row['INICIO'])
    termino = str(row['TERMINO'])
    atividade = str(row['ATIVIDADE']).replace('"', '\\"').replace("'", "\\'")
    local = str(row['LOCAL']).replace('"', '\\"').replace("'", "\\'")
    endereco = str(row['ENDEREÇO']).replace('"', '\\"').replace("'", "\\'")
    acesso = str(row['ACESSO']).replace('"', '\\"').replace("'", "\\'")
    contato = str(row['ARTICULAÇÃO']).replace('"', '\\"').replace("'", "\\'")
    
    # Create marker JavaScript code
    event_markers_js += f"""
// Event {idx}
const event{idx} = {{
    latitude: {row['latitude']},
    longitude: {row['longitude']},
    evento: "{evento}",
    data: "{data}",
    inicio: "{inicio}",
    termino: "{termino}",
    atividade: "{atividade}",
    local: "{local}",
    endereco: "{endereco}",
    acesso: "{acesso}",
    contato: "{contato}"
}};
events.push(event{idx});

const marker{idx} = L.marker([{row['latitude']}, {row['longitude']}], {{
    icon: L.icon({{
        iconUrl: 'https://unpkg.com/leaflet@1.7.1/dist/images/marker-icon.png',
        shadowUrl: 'https://unpkg.com/leaflet@1.7.1/dist/images/marker-shadow.png',
        iconSize: [25, 41],
        iconAnchor: [12, 41],
        popupAnchor: [1, -34],
        shadowSize: [41, 41]
    }})
}});

marker{idx}.bindPopup(`
    <div style="width: 250px">
        <h4>${{event{idx}.evento}}</h4>
        <p><b>Data:</b> ${{{data}}}</p>
        <p><b>Horário:</b> ${{{inicio}}} - ${{{termino}}}</p>
        <p><b>Atividade:</b> ${{{atividade}}}</p>
        <p><b>Local:</b> ${{{local}}}</p>
        <p><b>Endereço:</b> ${{{endereco}}}</p>
        <p><b>Acesso:</b> ${{{acesso}}}</p>
        <p><b>Articulação</b> ${{{contato}}}</p>
    </div>
`);
marker{idx}.bindTooltip("{evento} - {data}");
marker{idx}.addTo(map);

markers.push({{
    marker: marker{idx},
    date: "{data}"
}});
"""

# Insert the event markers JavaScript into the HTML
html_content = html_content.replace("// Add event data and markers", event_markers_js)

# Write the HTML file
with open('index.html', 'w', encoding='utf-8') as f:
    f.write(html_content)
print("Interactive index.html created successfully")

# Create iCalendar file
cal = Calendar()
cal.add('prodid', '-//City Nature Challenge 2025//São Paulo//BR')
cal.add('version', '2.0')
cal.add('x-wr-calname', 'City Nature Challenge 2025: São Paulo')
cal.add('description', 'Eventos do City Nature Challenge 2025 em São Paulo e Região Metropolitana')

# Add events to calendar
for idx, row in df_with_coords.iterrows():
    event = Event()
    
    # Set event summary (title)
    event.add('summary', f"{row['EVENTO']} - {row['ATIVIDADE']}")
    
    # Set event description
    description = f"Local: {row['LOCAL']}\nEndereço: {row['ENDEREÇO']}\nAcesso: {row['ACESSO']}\nArticulação {row['ARTICULAÇÃO']}"
    event.add('description', description)
    
    # Set event location
    event.add('location', str(row['ENDEREÇO']))
    
    # Parse date and time
    try:
        event_date = datetime.strptime(str(row['DATA']), '%d/%m/%Y')
        
        # Parse start time
        start_time_match = re.search(r'(\d+)h', str(row['INICIO']))
        start_hour = int(start_time_match.group(1)) if start_time_match else 9
        if start_hour >= 24:  # Handle invalid hours
            start_hour = 9
        
        # Parse end time
        end_time_match = re.search(r'(\d+)h', str(row['TERMINO']))
        end_hour = int(end_time_match.group(1)) if end_time_match else start_hour + 1
        if end_hour >= 24:  # Handle invalid hours
            end_hour = start_hour + 1
        if end_hour <= start_hour:  # Ensure end time is after start time
            end_hour = start_hour + 1
        
        # Create datetime objects for start and end
        start_datetime = datetime(event_date.year, event_date.month, event_date.day, start_hour, 0, 0)
        end_datetime = datetime(event_date.year, event_date.month, event_date.day, end_hour, 0, 0)
        
        # Add to event
        event.add('dtstart', start_datetime)
        event.add('dtend', end_datetime)
        
        # Add unique identifier
        event.add('uid', f"cnc2025-sp-{idx}@citynaturechallenge.org")
        
        # Add to calendar
        cal.add_component(event)
    except Exception as e:
        print(f"Error adding event {idx} to calendar: {e}")

# Write the iCalendar file
with open('city_nature_challenge_2025.ics', 'wb') as f:
    f.write(cal.to_ical())
print("iCalendar file (city_nature_challenge_2025.ics) created successfully")

print("\nAll files have been created successfully:")
print("1. eventos_2025_with_coords.tsv - Original data with coordinates")
print("2. map.html - Basic interactive map")
print("3. index.html - Enhanced interactive map with date filtering")
print("4. city_nature_challenge_2025.ics - Calendar file for importing into calendar applications")

Creating map with 22 locations (out of 22 total events)
Available dates: ['25/04/2025', '26/04/2025', '27/04/2025', '28/04/2025']
Interactive map saved to map.html
Interactive index.html created successfully
iCalendar file (city_nature_challenge_2025.ics) created successfully

All files have been created successfully:
1. eventos_2025_with_coords.tsv - Original data with coordinates
2. map.html - Basic interactive map
3. index.html - Enhanced interactive map with date filtering
4. city_nature_challenge_2025.ics - Calendar file for importing into calendar applications


In [45]:
# Create an interactive map with the geocoded locations
# This will also generate an HTML file that can be used for GitHub Pages

import folium
import pandas as pd
from datetime import datetime
import os
from icalendar import Calendar, Event
import pytz
import re

# Load the data with coordinates
df_with_coords = pd.read_csv("eventos_2025_with_coords.tsv", sep="\t")

# Filter out rows with missing coordinates
df_mapped = df_with_coords.dropna(subset=['latitude', 'longitude']).copy()
print(f"Creating map with {len(df_mapped)} locations (out of {len(df_with_coords)} total events)")

# Create a base map centered on São Paulo
map_center = [-23.55, -46.63]  # São Paulo coordinates
m = folium.Map(location=map_center, zoom_start=10, tiles="OpenStreetMap")

# Add markers for each location
for idx, row in df_mapped.iterrows():
    # Create popup content with event details
    popup_content = f"""
    <div style="width: 250px">
        <h4>{row['EVENTO']}</h4>
        <p><b>Data:</b> {row['DATA']}</p>
        <p><b>Horário:</b> {row['INICIO']} - {row['TERMINO']}</p>
        <p><b>Atividade:</b> {row['ATIVIDADE']}</p>
        <p><b>Local:</b> {row['LOCAL']}</p>
        <p><b>Endereço:</b> {row['ENDEREÇO']}</p>
        <p><b>Acesso:</b> {row['ACESSO']}</p>
        <p><b>Articulação</b> {row['ARTICULAÇÃO']}</p>
    </div>
    """
    
    # Add marker with popup
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_content, max_width=300),
        tooltip=f"{row['EVENTO']} - {row['DATA']}",
        icon=folium.Icon(color='green', icon='leaf', prefix='fa')
    ).add_to(m)

# Save the map to an HTML file
map_file = "map.html"
m.save(map_file)
print(f"Interactive map saved to {map_file}")

# Create a standalone HTML file with embedded map and date filtering
# Get unique dates for filtering (as strings to avoid serialization issues)
unique_dates = sorted(df_mapped['DATA'].unique())
print(f"Available dates: {unique_dates}")

# Create the HTML content with embedded data
html_content = f"""
<!DOCTYPE html>
<html lang="pt-BR">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>City Nature Challenge 2025: São Paulo e Região Metropolitana</title>
    <link rel="stylesheet" href="https://unpkg.com/leaflet@1.7.1/dist/leaflet.css" />
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            line-height: 1.6;
        }}
        .container {{
            max-width: 1200px;
            margin: 0 auto;
            padding: 20px;
        }}
        header {{
            background-color: #4CAF50;
            color: white;
            text-align: center;
            padding: 1em;
        }}
        #map {{
            height: 600px;
            width: 100%;
            margin-bottom: 20px;
        }}
        .controls {{
            margin-bottom: 20px;
            padding: 10px;
            background-color: #f5f5f5;
            border-radius: 5px;
        }}
        .btn {{
            background-color: #4CAF50;
            color: white;
            border: none;
            padding: 8px 16px;
            text-align: center;
            text-decoration: none;
            display: inline-block;
            font-size: 14px;
            margin: 4px 2px;
            cursor: pointer;
            border-radius: 4px;
        }}
        select {{
            padding: 8px;
            border-radius: 4px;
            border: 1px solid #ddd;
        }}
        footer {{
            background-color: #f5f5f5;
            text-align: center;
            padding: 1em;
            margin-top: 20px;
        }}
    </style>
</head>
<body>
    <header>
        <h1>City Nature Challenge 2025: São Paulo e Região Metropolitana</h1>
    </header>
    
    <div class="container">

        <h2>Eventos do City Nature Challenge 2025 na cidade de São Paulo e Região Metropolitana</h2>

        <div class="description">
            <p>Bem vindos ao City Nature Challenge 2025, também conhecido como Desafio Natureza nas Cidades! Por mais um ano estaremos competindo com cidades do mundo todo com o intuito de mostrar quão grande é a nossa biodiversidade! Seja nos parques, em trilhas, praças, ruas ou quintais, todos os registros são importantes e bem vindos! Então preparem seus celulares, suas câmeras e seus olhos, pois o desafio vai começar!</p>
        </div>

        <div class="description">
            <p>Para mais informações sobre o evento e a organização, acesse o projeto no <a href="https://www.inaturalist.org/projects/city-nature-challenge-2025-sao-paulo-e-regiao-metropolitana" target="_blank" style="color: #4CAF50; text-decoration: underline;">iNaturalist</a>.</p>
        </div>

        
        <h2> Note: este NÃO é o site oficial do evento, confira as informações antes de ir e boas observações!</h2>
        
        <div class="controls">
            <label for="date-filter">Filtrar por data: </label>
            <select id="date-filter">
                <option value="all">Todas as datas</option>
                {' '.join([f'<option value="{date}">{date}</option>' for date in unique_dates])}
            </select>
            <button class="btn" id="reset-filter">Mostrar Todos</button>
            <a href="city_nature_challenge_2025.ics" download class="btn" style="text-decoration: none;">Baixar Calendário (.ics)</a>
        </div>
        
        <div id="map"></div>
    </div>
    
    <footer>
        <p> Feito com carinho por <a href="https://tiago.bio.br" target="_blank" style="color: #4CAF50; text-decoration: underline;">Tiago</a>, da <a href="https://redenaturalista.org" target="_blank" style="color: #4CAF50; text-decoration: underline;">Rede Brasileira de Naturalistas</a> para o 2025 City Nature Challenge - São Paulo e Região Metropolitana.</p>
    </footer>

    <script src="https://unpkg.com/leaflet@1.7.1/dist/leaflet.js"></script>
    <script>
        // Initialize the map
        const map = L.map('map').setView([{map_center[0]}, {map_center[1]}], 10);
        
        // Add OpenStreetMap tiles
        L.tileLayer('https://{{s}}.tile.openstreetmap.org/{{z}}/{{x}}/{{y}}.png', {{
            attribution: '&copy; <a href="https://www.openstreetmap.org/copyright" target="_blank" style="color: #4CAF50; text-decoration: underline;">OpenStreetMap</a> contributors'
        }}).addTo(map);
        
        // Define event data directly in the script
        const events = [
            {', '.join([f"""{{
                latitude: {row['latitude']},
                longitude: {row['longitude']},
                evento: "{str(row['EVENTO']).replace('"', '\\"')}",
                data: "{str(row['DATA'])}",
                inicio: "{str(row['INICIO'])}",
                termino: "{str(row['TERMINO'])}",
                atividade: "{str(row['ATIVIDADE']).replace('"', '\\"')}",
                local: "{str(row['LOCAL']).replace('"', '\\"')}",
                endereco: "{str(row['ENDEREÇO']).replace('"', '\\"')}",
                acesso: "{str(row['ACESSO']).replace('"', '\\"')}",
                contato: "{str(row['ARTICULAÇÃO']).replace('"', '\\"')}"
            }}""" for idx, row in df_mapped.iterrows()])}
        ];
        
        // Add markers for each event
        const markers = [];
        
        events.forEach(event => {{
            const marker = L.marker([event.latitude, event.longitude], {{
                icon: L.icon({{
                    iconUrl: 'https://cdn.rawgit.com/pointhi/leaflet-color-markers/master/img/marker-icon-green.png',
                    shadowUrl: 'https://cdnjs.cloudflare.com/ajax/libs/leaflet/0.7.7/images/marker-shadow.png',
                    iconSize: [25, 41],
                    iconAnchor: [12, 41],
                    popupAnchor: [1, -34],
                    shadowSize: [41, 41]
                }})
            }});
            
            const popupContent = `
                <div style="width: 250px">
                    <h4>${{event.evento}}</h4>
                    <p><b>Data:</b> ${{event.data}}</p>
                    <p><b>Horário:</b> ${{event.inicio}} - ${{event.termino}}</p>
                    <p><b>Atividade:</b> ${{event.atividade}}</p>
                    <p><b>Local:</b> ${{event.local}}</p>
                    <p><b>Endereço:</b> ${{event.endereco}}</p>
                    <p><b>Acesso:</b> ${{event.acesso}}</p>
                    <p><b>Articulação</b> ${{event.contato}}</p>
                </div>
            `;
            
            marker.bindPopup(popupContent);
            marker.bindTooltip(`${{event.evento}} - ${{event.data}}`);
            marker.addTo(map);
            
            markers.push({{
                marker: marker,
                date: event.data
            }});
        }});
        
        // Filter events by date
        document.getElementById('date-filter').addEventListener('change', function() {{
            const selectedDate = this.value;
            
            markers.forEach(item => {{
                if (selectedDate === 'all' || item.date === selectedDate) {{
                    map.addLayer(item.marker);
                }} else {{
                    map.removeLayer(item.marker);
                }}
            }});
        }});
        
        // Reset filter button
        document.getElementById('reset-filter').addEventListener('click', function() {{
            document.getElementById('date-filter').value = 'all';
            markers.forEach(item => {{
                map.addLayer(item.marker);
            }});
        }});
    </script>
</body>
</html>
"""

# Write the HTML file
with open('index.html', 'w', encoding='utf-8') as f:
    f.write(html_content)
print("Interactive index.html created successfully")

# Create iCalendar file
cal = Calendar()
cal.add('prodid', '-//City Nature Challenge 2025//São Paulo//BR')
cal.add('version', '2.0')
cal.add('x-wr-calname', 'City Nature Challenge 2025: São Paulo')
cal.add('description', 'Eventos do City Nature Challenge 2025 em São Paulo e Região Metropolitana')

# Add events to calendar
for idx, row in df_with_coords.iterrows():
    try:
        event = Event()
        
        # Set event summary (title)
        event.add('summary', f"{row['EVENTO']} - {row['ATIVIDADE']}")
        
        # Set event description
        description = f"Local: {row['LOCAL']}\nEndereço: {row['ENDEREÇO']}\nAcesso: {row['ACESSO']}\nArticulação {row['ARTICULAÇÃO']}"
        event.add('description', description)
        
        # Set event location
        event.add('location', str(row['ENDEREÇO']))
        
        # Parse date and time
        try:
            event_date = datetime.strptime(str(row['DATA']), '%d/%m/%Y')
            
            # Parse start time
            start_time_match = re.search(r'(\d+)h', str(row['INICIO']))
            start_hour = int(start_time_match.group(1)) if start_time_match else 9
            if start_hour >= 24:  # Handle invalid hours
                start_hour = 9
            
            # Parse end time
            end_time_match = re.search(r'(\d+)h', str(row['TERMINO']))
            end_hour = int(end_time_match.group(1)) if end_time_match else start_hour + 1
            if end_hour >= 24:  # Handle invalid hours
                end_hour = start_hour + 1
            if end_hour <= start_hour:  # Ensure end time is after start time
                end_hour = start_hour + 1
            
            # Create datetime objects for start and end
            start_datetime = datetime(event_date.year, event_date.month, event_date.day, start_hour, 0, 0)
            end_datetime = datetime(event_date.year, event_date.month, event_date.day, end_hour, 0, 0)
            
            # Add to event
            event.add('dtstart', start_datetime)
            event.add('dtend', end_datetime)
            
            # Add unique identifier
            event.add('uid', f"cnc2025-sp-{idx}@citynaturechallenge.org")
            
            # Add to calendar
            cal.add_component(event)
        except Exception as e:
            print(f"Warning: Could not parse date/time for event {idx}: {e}")
            # If there's an error parsing the time, skip this event
            continue
    except Exception as e:
        print(f"Error adding event {idx} to calendar: {e}")

# Write the iCalendar file
with open('city_nature_challenge_2025.ics', 'wb') as f:
    f.write(cal.to_ical())
print("iCalendar file (city_nature_challenge_2025.ics) created successfully")

# Verify all files exist
files_created = [
    "eventos_2025_with_coords.tsv",
    "map.html",
    "index.html",
    "city_nature_challenge_2025.ics"
]

print("\nVerifying created files:")
for file in files_created:
    if os.path.exists(file):
        file_size = os.path.getsize(file)
        print(f"✓ {file} - {file_size} bytes")
    else:
        print(f"✗ {file} - Not found!")

print("\nAll files have been created successfully!")
print("1. eventos_2025_with_coords.tsv - Original data with coordinates")
print("2. map.html - Basic interactive map")
print("3. index.html - Enhanced interactive map with date filtering")
print("4. city_nature_challenge_2025.ics - Calendar file for importing into calendar applications")

print("\nTo use these files with GitHub Pages:")
print("1. Upload all files to your GitHub repository")
print("2. Enable GitHub Pages in repository settings")
print("3. The index.html file will be served as the main page")

Creating map with 22 locations (out of 22 total events)
Interactive map saved to map.html
Available dates: ['25/04/2025', '26/04/2025', '27/04/2025', '28/04/2025']
Interactive index.html created successfully
iCalendar file (city_nature_challenge_2025.ics) created successfully

Verifying created files:
✓ eventos_2025_with_coords.tsv - 6476 bytes
✓ map.html - 44959 bytes
✓ index.html - 20051 bytes
✓ city_nature_challenge_2025.ics - 11807 bytes

All files have been created successfully!
1. eventos_2025_with_coords.tsv - Original data with coordinates
2. map.html - Basic interactive map
3. index.html - Enhanced interactive map with date filtering
4. city_nature_challenge_2025.ics - Calendar file for importing into calendar applications

To use these files with GitHub Pages:
1. Upload all files to your GitHub repository
2. Enable GitHub Pages in repository settings
3. The index.html file will be served as the main page
